This is the Import statements required for the data

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
pd.Series.__unicode__ = pd.Series.to_string
import numpy as np

In [2]:
import plotting

/Users/Chris/anaconda/lib/python3.6/site-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [3]:
import copy

In [4]:
import utils

In [5]:
df=pd.read_csv("./census2011.csv")

In [6]:
newdf = copy.deepcopy(df)
for field in newdf:
    newdf[field] = newdf[field].apply(lambda value: utils.data_translation(field,value))

Basic Requirement Number 2

In [7]:
#Basic Requirement 2.1
print("The number of records is " + str(len(df)))

The number of records is 569741


In [8]:
#Basic Requirement 2.2
print("All Data Types:\n" + str(df.dtypes))

All Data Types:
Person ID                     int64
Region                       object
Residence Type               object
Family Composition            int64
Population Base               int64
Sex                           int64
Age                           int64
Marital Status                int64
Student                       int64
Country of Birth              int64
Health                        int64
Ethnic Group                  int64
Religion                      int64
Economic Activity             int64
Occupation                    int64
Industry                      int64
Hours worked per week         int64
Approximated Social Grade     int64


In [9]:
#Basic Requirement 2.3
for column in newdf:
    if (column != "Person ID"):
        print(str(column)+ " Counts\n" +str(newdf[column].value_counts()) + "\n")

Region Counts
South East                  88084
London                      83582
North West                  71436
East of England             59411
West Midlands               56875
South West                  53774
Yorkshire and the Humber    53471
East Midlands               45782
Wales                       30977
North East                  26349

Residence Type Counts
Not Resident    559087
Resident         10654

Family Composition Counts
Married/Civil Partnership      300962
Not in Family                   96690
Cohabiting Couple Family        72641
Lone Parent Family (Female)     64519
No Code Required                18851
Lone Parent Family (Male)        9848
Other Related Family             6230

Population Base Counts
Usual Resident                   561040
Student Living Away From Home      6730
Short-term Resident                1971

Sex Counts
Female    289172
Male      280569

Age Counts
0 to 15        106832
35 to 44        78641
45 to 54        77388
25 to 34        

Plotting Graph Basic Requirements 3

In [ ]:
# Number of Records per region
plotting.plot_bar_group(
        newdf,
        'Region',
        'Frequencies of Resident Regions in 2011 Census',
        'Region',
        'Frequency',
    )

plt.tight_layout()
plt.show()

In [ ]:
# Number of records per occupation
plotting.plot_bar_group(
        newdf,
        'Occupation',
        'Frequencies of Resident Occupations in 2011 Census',
        'Occupation',
        'Frequency',
    )

plt.tight_layout()
plt.show()

In [ ]:
#Pie Chart for Distrubition of the sample by age
plotting.plot_pie_group(
        newdf,
        'Age',
        'Proportions of Resident Ages in 2011 Census',
    )

plt.tight_layout()
plt.show()

In [ ]:
#Pie Chart for Distrubition of the sample by economic activity
plotting.plot_pie_group(
        newdf,
        'Economic Activity',
        'Proportions of Resident Economic Activity in 2011 Census',
    )

plt.tight_layout()
plt.show()

Easy Extension 1

In [ ]:
#Easy Extension 1 part 1 Using crosstab
dfeasy11 = pd.crosstab(newdf['Region'], newdf['Industry'])
dfeasy11

In [ ]:
#Easy Extension 1 part 2 using crosstab
dfeasy12 = pd.crosstab(newdf['Occupation'], newdf['Approximated Social Grade'])
dfeasy12

In [ ]:
#Easy 1 part 1 using GroupBy
easy11 = newdf.groupby(['Region','Industry'])['Person ID'].count()
easy11 = pd.Series.to_frame(easy11)
easy11.columns.values[0] = 'Counts'
easy11

In [ ]:
#Easy 1 part 2 using GroupBy
easy12 = newdf.groupby(['Occupation','Approximated Social Grade'])['Person ID'].count()
easy12 = pd.Series.to_frame(easy12)
easy12.columns.values[0] = 'Counts'
easy12

Easy Extension 2 Parts 1 and 2 -Part 3 was filtered in the Dataset to begin with

In [ ]:
#Filtering the Dataset to be economic code from 1-4
ea = df[df["Economic Activity"].isin(range(1, 5))]
for field in ea:
    ea[field] = ea[field].apply(lambda value: utils.data_translation(field,value))

In [ ]:
# Easy 2 Part 1 using CrossTab
easy21 = pd.crosstab(index = (ea)['Region'], columns = 'count')
easy21 = easy21.sort_values(['count'],ascending = 0)
easy21

In [ ]:
# Easy 2 Part 2 using CrossTab
easy22 = pd.crosstab(index = (ea)['Age'], columns = 'count')
easy22 = easy22.sort_values(['count'],ascending = 0)
easy22

Here Follows Testing Code (ignore unless you notice snippets of code you want to use i.e ipywidgets): 

In [ ]:
newdf = df
newdf

In [ ]:
#for index, row in newdf.iterrows():
 #   newdf.set_value('Residence Type', index, 10)
    #print(str(row['Residence Type']))
#newdf
newdf.loc[0,'Residence Type'] = 10

In [ ]:
newdf

In [ ]:
for column in newdf:
    print(column)

In [ ]:
for index, row in newdf.iterrows():
    #print(str(row))

In [ ]:
row = 0
column = 'Residence Type'
df.loc[row,column] = 10
df

In [ ]:
#funcname = lambda x: x = 10

df["Residence Type"].apply(lambda x: str(x).lower())

# for column in df:
#     print(column)
#     if (column == 'Residence Type'):
        
df

In [ ]:
easy11 = df.groupby(['Region','Industry'])['Person ID'].count()

In [ ]:
easy11 = df

In [ ]:
#easy11 = pd.Series.to_frame(easy11)
#pd.crosstab(easy1['Region'], easy1['Industry'])
easy11 = df.groupby('Region').agg({"Industry":})

In [ ]:
easy11

In [ ]:
easy11 = df.groupby(['Region','Industry'])['Person ID'].count()
easy11 = pd.Series.to_frame(easy11)
easy11.columns.values[0] = 'Counts'
easy11

In [ ]:
easy12 = df.groupby(['Occupation','Approximated Social Grade'])['Person ID'].count()
easy12 = pd.Series.to_frame(easy12)
easy12.columns.values[0] = 'Counts'
easy12

In [ ]:
pd.crosstab(df['Occupation'], df['Approximated Social Grade'])

In [ ]:
pd.crosstab(df['Region'], df['Industry'])

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
def f(x):
    return x

In [ ]:
interact(f, x=10)

In [ ]:
interact(f, x=True)

In [ ]:
interact(f, x=['one','two']);

In [ ]:
print(1)

In [ ]:
len(df)

In [ ]:
ea = df[df["Economic Activity"].isin(range(0, 5))]

In [ ]:
easy21 = pd.crosstab(index = (ea)['Region'], columns = 'count')
easy21

In [ ]:
easy22 = pd.crosstab(index = (ea)['Age'], columns = 'count')
easy22

In [ ]:
#counting the number of records
print("number of records = " + str(df['Person ID'].count()))

In [ ]:
df['Residence Type'].value_counts()

In [ ]:
print(str(df.groupby('Economic Activity')))